In [1]:
import numpy as np

from nn import Module
from init import normal
from arithmetic import Mean, Sub, Pow, Add, Mul

In [2]:
class MSELoss(Module):
    
    def __init__(self):
        self.sub = Sub()
        self.pow = Pow()
        self.mean = Mean() 
    
    def forward(self, x, x_pred):
        y = self.sub(x, x_pred)
        y = self.pow(y, 2)
        y = self.mean(y)
        return y
    
    def backward(self, lr):
        dy = self.mean.backward(lr)
        dy = self.pow.backward(dy)
        dx, dx_pred = self.sub.backward(dy) 
        return dx_pred 

In [3]:
class Linear2(Module):
    
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        self.in_features = in_features
        self.out_features = out_features
        
        # Might want to make this additional dimension abstract 
        self.weights = normal((out_features, in_features))
        self.bias = normal((out_features, 1)) if bias else None
        
        self.add = Add() 
        self.mul = Mul() 
        
    def forward(self, x):
        y = self.mul(self.weights, x)
        y = self.add(self.bias, y)
        return y
    
    def backward(self, dy, lr=0.1):
        db, dy = self.add.backward(dy)
        dw, dx = self.mul.backward(dy)
        
        self.bias -= db * lr
        self.weights -= dw * lr
        return dx 
        

In [4]:
class MSE(Module): 
    
    def __init__(self):
        self.x = None 
        self.x_pred = None
    
    def forward(self, x, x_pred):
        self.x = x 
        self.x_pred = x_pred
        return np.mean((x - x_pred)**2, keepdims=True)
    
    def backward(self, dy=1): 
        return 2 * (self.x_pred - self.x) / len(self.x) * dy

In [9]:
i = 2
o = 3
lr = np.array([0.1])

criterion = MSELoss()
model = Linear2(i, o, bias=True)
y = np.ones((o, 1)) * 0.25
x = np.ones((i, 1))

In [10]:
model(x)

array([[-2.66225232],
       [ 1.24078871],
       [-2.00418404]])

In [11]:
for i in range(100):
    y_hat = model(x)
    loss = criterion(y, y_hat)
#     print(criterion.backward())
#     print(f"{loss=}", end='\r')
    model.backward(criterion.backward(lr))

In [12]:
model(x)

array([[0.25],
       [0.25],
       [0.25]])

In [ ]:
np.ones(5).reshape(-1, 1) @ np.arange(3).reshape(-1, 1).T

In [ ]:
class Linear(Module):
    
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        self.in_features = in_features
        self.out_features = out_features
        self.weights = normal((out_features, in_features))
        self.bias = normal(out_features) if bias else None
        self.x = None
        
    def forward(self, x):
        self.x = x
        if self.bias is not None: 
            return self.weights @ x + self.bias 
        else:
            return self.weights @ x
    
    def backward(self, dy, lr=0.01):
        # dy / dx = w
        dx = self.weights.T @ dy 
        
        # dz / dw =  x 
        dw = dy.reshape(-1, 1) * self.x
        self.weights -= dw * lr
        
        if self.bias is not None: 
            # dy / db = 1 
            db = 1 * dy
            self.bias -= db * lr 
            
        return dx 

In [ ]:
class MLP(Module):
    
    def __init__(self, in_features: int, hidden_features: int, out_features: int, bias: bool = True):
        self.lin1 = Linear(in_features, hidden_features, bias)
        self.act1 = Sigmoid()
        self.lin2 = Linear(hidden_features, out_features, bias)
    
    def forward(self, x):
        h = self.lin1(x)
        h = self.act1(h)
        h = self.lin2(h)
        return h 
    
    def backward(self, dy=1):
        dy = self.lin2.backward(dy=dy)
        dy = self.act1.backward(dy=dy)
        dy = self.lin1.backward(dy=dy)
        return dy 

In [ ]:
class MSE(Module): 
    
    def __init__(self):
        self.x = None 
        self.x_hat = None
    
    def forward(self, x, x_hat):
        self.x = x 
        self.x_hat = x_hat
        return np.mean((x - x_hat)**2, keepdims=True)
    
    def backward(self, dy=1): 
        return 2 * (self.x_hat - self.x) / len(self.x) * dy 

In [ ]:
class ReLU(Module):
    
    def __init__(self):
        self.x = None
    
    def forward(self, x):
        self.x = x
        return x * (0 < x)
    
    def backward(self, dy):
        return (0 < self.x) * dy

In [ ]:
class Sigmoid(Module):
    
    def __init__(self):
        self.x = None
    
    def forward(self, x):
        self.x = x
        return np.exp(x) / (1 + np.exp(x))
    
    def backward(self, dy): 
        return (np.exp(self.x) / (1 + np.exp(self.x))**2).T * dy

In [ ]:
i = 2
o = 15
h = 10

criterion = MSE()
model = MLP(i, h, o, nonlin=Sigmoid(), bias=True)
y = np.ones(o) * 0.25
x = np.ones(i)

In [ ]:
for i in range(10000):
    y_hat = model(x)
    loss = criterion(y, y_hat)
#     print(criterion.backward())
    model.backward(criterion.backward())

In [ ]:
model(x)